# 영화 추천: ranking 단계

실제 추천 시스템은 종종 두 단계로 구성됩니다.

1. 검색 단계(Retrieval Phase)에서는 가능한 모든 후보 중에서 수백개의 초기 후보 집합을 선택합니다. 이 모델의 주요 목적은 사용자가 관심이 없는 모든 후보를 효율적으로 제거하는 것입니다. 검색 모델은 수백만 개의 후보를 처리할 수 있으므로 계산적으로 효율적이어야 합니다.  

2. 순위 단계(Ranking Phase)에서는 검색 모델의 출력을 가져와서 가능한 한 가장 좋은 추천 항목을 선택하도록 미세 조정합니다. 이 작업은 사용자가 관심을 가질 만한 항목 집합을 가능한 후보의 최종 목록으로 좁히는 것입니다.

이 notebook 은 순위 단계 입니다.  

다음을 수행합니다.

1. 데이터를 가져와 훈련 세트와 테스트 세트로 나눕니다.
2. 순위 모델을 구현합니다.
3. 피팅하고 평가합니다.


In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 1.5 MB/s 
     |████████████████████████████████| 462 kB 39.2 MB/s 
     |████████████████████████████████| 4.2 MB 5.6 MB/s 


In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

## Preparing the dataset

[retrieval](basic_retrieval) 과 동일한 데이터를 사용합니다. 이번에는 등급(explicit feedback)도 유지합니다. 이는 우리가 예측하려는 target 입니다.

In [3]:
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-ratings/0.1.0.incomplete5EEEU9/movielens-train.tfrecord*...…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-ratings/0.1.0. Subsequent calls will reuse this data.


이전과 마찬가지로 train/test를 80% 대 20%로 분할합니다.

In [5]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)
len(train), len(test)

(80000, 20000)

## 모델 구현

In [6]:
user_ids_vocabulary = tf.keras.layers.StringLookup()
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

In [8]:
movie_titles_vocabulary = tf.keras.layers.StringLookup()
movie_titles_vocabulary.adapt(ratings.map(lambda x: x["movie_title"]))

In [9]:
user_ids_vocabulary.vocabulary_size(), movie_titles_vocabulary.vocabulary_size()

(944, 1665)

### Architecture

랭킹 모델은 검색 모델처럼 효율성 제약에 직면하지 않으므로 아키텍처를 선택할 때 조금 더 자유로워집니다.

여러 스택의 dense layer로 구성된 모델은 task 순위 지정을 위한 비교적 일반적인 아키텍처입니다. 다음과 같이 구현할 수 있습니다.

In [10]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # user에 대한 임베딩 계산
    self.user_embeddings = tf.keras.Sequential([
      user_ids_vocabulary,
      tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), embedding_dimension)
    ])

    # movie에 대한 임베딩 계산
    self.movie_embeddings = tf.keras.Sequential([
      movie_titles_vocabulary,
      tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), embedding_dimension)
    ])

    # prediction 계산
    self.ratings = tf.keras.Sequential([
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      tf.keras.layers.Dense(1)   # Make rating predictions
  ])
    
  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

이 모델은 user ID와 movie title을 가져와 예상 등급을 출력합니다.

In [11]:
RankingModel()((["42"], ["One Flew Over the Cuckoo's Nest (1975)"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.00049353]], dtype=float32)>

### Loss and metrics

다음 구성 요소는 모델을 훈련하는 데 사용되는 손실입니다. TFRS에는 이를 쉽게 하기 위한 여러 손실 계층과 작업이 있습니다.

이 경우 손실 함수와 메트릭 계산을 함께 묶는 편리한 래퍼인 `Ranking` 작업 개체를 사용합니다.

평점을 예측하기 위해 `MeanSquaredError` Keras 손실과 함께 사용할 것입니다.

In [12]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

task 자체는 true 와 predicted 를 인수로 받아 계산된 손실을 반환하는 Keras 계층입니다. 이를 사용하여 모델의 훈련 루프를 구현합니다.

### The full model

이제 모든 것을 하나의 모델로 통합할 수 있습니다. TFRS는 빌딩 모델을 간소화하는 base 모델 클래스(`tfrs.models.Model`)를 노출합니다. feature를 제공하고 손실 값을 반환합니다.

그러면 base model이 우리 모델에 맞는 적절한 훈련 루프를 생성합니다.

In [13]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")
    
    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

## Fitting and evaluating

In [14]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

훈련 및 평가 데이터를 섞고 일괄 처리하고 캐시합니다.

In [15]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

 모델을 훈련합니다.

In [16]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 5s 99ms/step - root_mean_squared_error: 2.1730 - loss: 4.3345 - regularization_loss: 0.0000e+00 - total_loss: 4.3345
Epoch 2/3
10/10 [==============================] - 1s 58ms/step - root_mean_squared_error: 1.1201 - loss: 1.2542 - regularization_loss: 0.0000e+00 - total_loss: 1.2542
Epoch 3/3
10/10 [==============================] - 1s 58ms/step - root_mean_squared_error: 1.1124 - loss: 1.2369 - regularization_loss: 0.0000e+00 - total_loss: 1.2369


In [17]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 4s 33ms/step - root_mean_squared_error: 1.1062 - loss: 1.2189 - regularization_loss: 0.0000e+00 - total_loss: 1.2189


{'loss': 1.19796884059906,
 'regularization_loss': 0,
 'root_mean_squared_error': 1.1061803102493286,
 'total_loss': 1.19796884059906}

## Testing the ranking model

이제 영화 세트에 대한 예측을 계산하여 ranking model을 테스트한 다음 예측을 기반으로 이러한 영화의 순위를 지정할 수 있습니다.


In [18]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
M*A*S*H (1970): [[3.566162]]
Dances with Wolves (1990): [[3.5358002]]
Speed (1994): [[3.5338762]]
